In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.metrics import mean_absolute_error, mean_squared_error,accuracy_score, f1_score, recall_score
import sys
import warnings
from itertools import cycle
warnings.filterwarnings("ignore")
np.random.seed(40)
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import pickle
#Calling the preprocessor pickle
with open('prepic.pkl', 'rb') as f:
    pre = pickle.load(f)

mlflow.set_tracking_uri('http://127.0.0.1:5000')

train = pd.read_csv(r"D:\mlopset\train_set_ml.csv")

test = pd.read_csv(r"D:\mlopset\test_set_ml.csv")

import xgboost as xg

def eval_metrics(actual, pred):
    accuracy =  accuracy_score(actual, pred)
    f1 = f1_score(actual, pred)
    recall = recall_score(actual, pred)
    return accuracy, f1, recall

train_x = train.drop(["status"], axis=1)
test_x = test.drop(["status"], axis=1)
train_y = train[["status"]]
test_y = test[["status"]]

train_x_tf = pre.transform(train_x)
test_x_tf = pre.transform(test_x)

# Define the experiment name
experiment_name = "loan_status_experiment"

# Check if the experiment exists
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    # If the experiment does not exist, create it
    mlflow.create_experiment(experiment_name)

# Set the experiment
mlflow.set_experiment(experiment_name)

# Defining the hyperparameters
eta_v = [0.01,0.03,0.1,0.2]
subsample_v = [0.5, .9, 1]
max_depth_v = [6,8,10]
gamma_v = [0.01,.1, .2, .3]
max_delta_step_v = [2,4,6,8]

for eta in eta_v:
    for subsample in subsample_v:
        for max_depth in max_depth_v:
            for gamma in gamma_v:
                for max_delta_step in max_delta_step_v:
                    with mlflow.start_run(run_name=f"eta_{eta}_subsample_{subsample}_max_depth_{max_depth}_gamma_{gamma}_max_delta_step{max_delta_step}"):
                        # Run XGB
                        xgb = xg.XGBClassifier(eta=eta, subsample=subsample, max_depth=max_depth, gamma=gamma, max_delta_step=max_delta_step, random_state=42)
                        xgb.fit(train_x_tf, train_y)
                        predicted_qualities = xgb.predict(test_x_tf)
                        (accuracy, f1, recall) = eval_metrics(test_y, predicted_qualities)
            
                        # Print out XGB model metrics
                        print(f"XGBoost Model (eta={eta:f}, subsample={subsample:f}, max_depth={max_depth:f}, gamma={gamma:f}, max_delta_step={max_delta_step:f}):")
                        print(f"  accuracy: {accuracy}")
                        print(f"  f1: {f1}")
                        print(f"  recall: {recall}")
                        
                        # Infer model signature
                        predictions = xgb.predict(train_x_tf)
                        signature = infer_signature(train_x_tf, predictions)
            
                        # Log mlflow attributes for mlflow UI
                        mlflow.log_param("eta", eta)
                        mlflow.log_param("subsample", subsample)
                        mlflow.log_param("max_depth", max_depth)
                        mlflow.log_param("gamma", gamma)
                        mlflow.log_param("max_delta_step", max_delta_step)
                        mlflow.log_metric("accuracy", accuracy)
                        mlflow.log_metric("f1", f1)
                        mlflow.log_metric("recall", recall)
                        mlflow.sklearn.log_model(xgb, "model", signature=signature)

# end current run
mlflow.end_run()

# Get experiment ID
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

# Search all runs in experiment_id
runs = mlflow.search_runs(experiment_ids=experiment_id)

# display runs dataframe
runs

XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.010000, max_delta_step=2.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.010000, max_delta_step=4.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.010000, max_delta_step=6.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.010000, max_delta_step=8.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, gamma=0.100000, max_delta_step=2.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=6.000000, g

XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=10.000000, gamma=0.300000, max_delta_step=4.000000):
  accuracy: 0.6989247311827957
  f1: 0.2631578947368421
  recall: 0.15625
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=10.000000, gamma=0.300000, max_delta_step=6.000000):
  accuracy: 0.6989247311827957
  f1: 0.2631578947368421
  recall: 0.15625
XGBoost Model (eta=0.010000, subsample=0.500000, max_depth=10.000000, gamma=0.300000, max_delta_step=8.000000):
  accuracy: 0.6989247311827957
  f1: 0.2631578947368421
  recall: 0.15625
XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=6.000000, gamma=0.010000, max_delta_step=2.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=6.000000, gamma=0.010000, max_delta_step=4.000000):
  accuracy: 0.6989247311827957
  f1: 0.2222222222222222
  recall: 0.125
XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=6.000000, gamm

XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=10.000000, gamma=0.200000, max_delta_step=6.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=10.000000, gamma=0.200000, max_delta_step=8.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=10.000000, gamma=0.300000, max_delta_step=2.000000):
  accuracy: 0.6881720430107527
  f1: 0.17142857142857143
  recall: 0.09375
XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=10.000000, gamma=0.300000, max_delta_step=4.000000):
  accuracy: 0.6989247311827957
  f1: 0.2222222222222222
  recall: 0.125
XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=10.000000, gamma=0.300000, max_delta_step=6.000000):
  accuracy: 0.6989247311827957
  f1: 0.2222222222222222
  recall: 0.125
XGBoost Model (eta=0.010000, subsample=0.900000, max_depth=10.000000, g

XGBoost Model (eta=0.010000, subsample=1.000000, max_depth=10.000000, gamma=0.100000, max_delta_step=8.000000):
  accuracy: 0.7096774193548387
  f1: 0.2702702702702703
  recall: 0.15625
XGBoost Model (eta=0.010000, subsample=1.000000, max_depth=10.000000, gamma=0.200000, max_delta_step=2.000000):
  accuracy: 0.6989247311827957
  f1: 0.2631578947368421
  recall: 0.15625
XGBoost Model (eta=0.010000, subsample=1.000000, max_depth=10.000000, gamma=0.200000, max_delta_step=4.000000):
  accuracy: 0.7096774193548387
  f1: 0.2702702702702703
  recall: 0.15625
XGBoost Model (eta=0.010000, subsample=1.000000, max_depth=10.000000, gamma=0.200000, max_delta_step=6.000000):
  accuracy: 0.7096774193548387
  f1: 0.2702702702702703
  recall: 0.15625
XGBoost Model (eta=0.010000, subsample=1.000000, max_depth=10.000000, gamma=0.200000, max_delta_step=8.000000):
  accuracy: 0.7096774193548387
  f1: 0.2702702702702703
  recall: 0.15625
XGBoost Model (eta=0.010000, subsample=1.000000, max_depth=10.000000, 

XGBoost Model (eta=0.030000, subsample=0.500000, max_depth=10.000000, gamma=0.100000, max_delta_step=6.000000):
  accuracy: 0.7849462365591398
  f1: 0.6153846153846154
  recall: 0.5
XGBoost Model (eta=0.030000, subsample=0.500000, max_depth=10.000000, gamma=0.100000, max_delta_step=8.000000):
  accuracy: 0.7849462365591398
  f1: 0.6153846153846154
  recall: 0.5
XGBoost Model (eta=0.030000, subsample=0.500000, max_depth=10.000000, gamma=0.200000, max_delta_step=2.000000):
  accuracy: 0.7741935483870968
  f1: 0.6037735849056604
  recall: 0.5
XGBoost Model (eta=0.030000, subsample=0.500000, max_depth=10.000000, gamma=0.200000, max_delta_step=4.000000):
  accuracy: 0.7634408602150538
  f1: 0.56
  recall: 0.4375
XGBoost Model (eta=0.030000, subsample=0.500000, max_depth=10.000000, gamma=0.200000, max_delta_step=6.000000):
  accuracy: 0.7634408602150538
  f1: 0.56
  recall: 0.4375
XGBoost Model (eta=0.030000, subsample=0.500000, max_depth=10.000000, gamma=0.200000, max_delta_step=8.000000):


...

XGBoost Model (eta=0.100000, subsample=1.000000, max_depth=8.000000, gamma=0.100000, max_delta_step=8.000000):
  accuracy: 0.7849462365591398
  f1: 0.6153846153846154
  recall: 0.5
XGBoost Model (eta=0.100000, subsample=1.000000, max_depth=8.000000, gamma=0.200000, max_delta_step=2.000000):
  accuracy: 0.7526881720430108
  f1: 0.6101694915254238
  recall: 0.5625
XGBoost Model (eta=0.100000, subsample=1.000000, max_depth=8.000000, gamma=0.200000, max_delta_step=4.000000):
  accuracy: 0.7741935483870968
  f1: 0.631578947368421
  recall: 0.5625
XGBoost Model (eta=0.100000, subsample=1.000000, max_depth=8.000000, gamma=0.200000, max_delta_step=6.000000):
  accuracy: 0.7741935483870968
  f1: 0.631578947368421
  recall: 0.5625
XGBoost Model (eta=0.100000, subsample=1.000000, max_depth=8.000000, gamma=0.200000, max_delta_step=8.000000):
  accuracy: 0.7741935483870968
  f1: 0.631578947368421
  recall: 0.5625
XGBoost Model (eta=0.100000, subsample=1.000000, max_depth=8.000000, gamma=0.300000, m

XGBoost Model (eta=0.200000, subsample=0.500000, max_depth=8.000000, gamma=0.100000, max_delta_step=2.000000):
  accuracy: 0.7956989247311828
  f1: 0.6779661016949152
  recall: 0.625
XGBoost Model (eta=0.200000, subsample=0.500000, max_depth=8.000000, gamma=0.100000, max_delta_step=4.000000):
  accuracy: 0.7956989247311828
  f1: 0.6779661016949152
  recall: 0.625
XGBoost Model (eta=0.200000, subsample=0.500000, max_depth=8.000000, gamma=0.100000, max_delta_step=6.000000):
  accuracy: 0.7956989247311828
  f1: 0.6779661016949152
  recall: 0.625
XGBoost Model (eta=0.200000, subsample=0.500000, max_depth=8.000000, gamma=0.100000, max_delta_step=8.000000):
  accuracy: 0.7956989247311828
  f1: 0.6779661016949152
  recall: 0.625
XGBoost Model (eta=0.200000, subsample=0.500000, max_depth=8.000000, gamma=0.200000, max_delta_step=2.000000):
  accuracy: 0.8064516129032258
  f1: 0.7
  recall: 0.65625
XGBoost Model (eta=0.200000, subsample=0.500000, max_depth=8.000000, gamma=0.200000, max_delta_ste

XGBoost Model (eta=0.200000, subsample=0.900000, max_depth=8.000000, gamma=0.010000, max_delta_step=4.000000):
  accuracy: 0.7849462365591398
  f1: 0.6153846153846154
  recall: 0.5
XGBoost Model (eta=0.200000, subsample=0.900000, max_depth=8.000000, gamma=0.010000, max_delta_step=6.000000):
  accuracy: 0.7849462365591398
  f1: 0.6153846153846154
  recall: 0.5
XGBoost Model (eta=0.200000, subsample=0.900000, max_depth=8.000000, gamma=0.010000, max_delta_step=8.000000):
  accuracy: 0.7849462365591398
  f1: 0.6153846153846154
  recall: 0.5
XGBoost Model (eta=0.200000, subsample=0.900000, max_depth=8.000000, gamma=0.100000, max_delta_step=2.000000):
  accuracy: 0.7741935483870968
  f1: 0.6181818181818182
  recall: 0.53125
XGBoost Model (eta=0.200000, subsample=0.900000, max_depth=8.000000, gamma=0.100000, max_delta_step=4.000000):
  accuracy: 0.7849462365591398
  f1: 0.6428571428571429
  recall: 0.5625
XGBoost Model (eta=0.200000, subsample=0.900000, max_depth=8.000000, gamma=0.100000, max

XGBoost Model (eta=0.200000, subsample=1.000000, max_depth=6.000000, gamma=0.300000, max_delta_step=6.000000):
  accuracy: 0.7419354838709677
  f1: 0.5555555555555556
  recall: 0.46875
XGBoost Model (eta=0.200000, subsample=1.000000, max_depth=6.000000, gamma=0.300000, max_delta_step=8.000000):
  accuracy: 0.7419354838709677
  f1: 0.5555555555555556
  recall: 0.46875
XGBoost Model (eta=0.200000, subsample=1.000000, max_depth=8.000000, gamma=0.010000, max_delta_step=2.000000):
  accuracy: 0.7419354838709677
  f1: 0.5862068965517242
  recall: 0.53125
XGBoost Model (eta=0.200000, subsample=1.000000, max_depth=8.000000, gamma=0.010000, max_delta_step=4.000000):
  accuracy: 0.7849462365591398
  f1: 0.6296296296296297
  recall: 0.53125
XGBoost Model (eta=0.200000, subsample=1.000000, max_depth=8.000000, gamma=0.010000, max_delta_step=6.000000):
  accuracy: 0.7849462365591398
  f1: 0.6296296296296297
  recall: 0.53125
XGBoost Model (eta=0.200000, subsample=1.000000, max_depth=8.000000, gamma=

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.recall,metrics.f1,metrics.accuracy,metrics.mae,...,params.max_delta_step,params.max_depth,params.eta,params.gamma,params.max_depthgb,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.log-model.history,tags.mlflow.runName,tags.mlflow.source.name
0,25534d5ffb2b48a98613eb98330b2ca9,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/25534d5ff...,2024-01-17 11:26:09.541000+00:00,2024-01-17 11:26:14.458000+00:00,0.46875,0.576923,0.763441,NaN,...,8,10,0.2,0.3,None,LOCAL,admin,"[{""run_id"": ""25534d5ffb2b48a98613eb98330b2ca9""...",eta_0.2_subsample_1_max_depth_10_gamma_0.3_max...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
1,74490058cdd34d5589190598072027df,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/74490058c...,2024-01-17 11:26:04.491000+00:00,2024-01-17 11:26:09.497000+00:00,0.46875,0.576923,0.763441,NaN,...,6,10,0.2,0.3,None,LOCAL,admin,"[{""run_id"": ""74490058cdd34d5589190598072027df""...",eta_0.2_subsample_1_max_depth_10_gamma_0.3_max...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
2,45a04b7639bb4c23b1828e93f7a86fbf,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/45a04b763...,2024-01-17 11:25:59.462000+00:00,2024-01-17 11:26:04.447000+00:00,0.46875,0.576923,0.763441,NaN,...,4,10,0.2,0.3,None,LOCAL,admin,"[{""run_id"": ""45a04b7639bb4c23b1828e93f7a86fbf""...",eta_0.2_subsample_1_max_depth_10_gamma_0.3_max...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
3,5acbe4da29ce49c99ec1039726564bf9,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/5acbe4da2...,2024-01-17 11:25:54.259000+00:00,2024-01-17 11:25:59.427000+00:00,0.50000,0.592593,0.763441,NaN,...,2,10,0.2,0.3,None,LOCAL,admin,"[{""run_id"": ""5acbe4da29ce49c99ec1039726564bf9""...",eta_0.2_subsample_1_max_depth_10_gamma_0.3_max...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
4,9818bdab6ac346c08588095da62af7f4,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/9818bdab6...,2024-01-17 11:25:49.125000+00:00,2024-01-17 11:25:54.210000+00:00,0.50000,0.603774,0.774194,NaN,...,8,10,0.2,0.2,None,LOCAL,admin,"[{""run_id"": ""9818bdab6ac346c08588095da62af7f4""...",eta_0.2_subsample_1_max_depth_10_gamma_0.2_max...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,6667be05f64a4a0dac36e1cb88ee6069,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/6667be05f...,2024-01-17 09:33:19.586000+00:00,2024-01-17 09:33:25.443000+00:00,NaN,NaN,NaN,0.311828,...,2,6,0.01,0.1,None,LOCAL,admin,"[{""run_id"": ""6667be05f64a4a0dac36e1cb88ee6069""...",eta_0.01_subsample_0.5_max_depth_6_gamma_0.1_m...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
1152,f77ef7565cc24c7d8a590f7af3c55a7a,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/f77ef7565...,2024-01-17 09:33:13.923000+00:00,2024-01-17 09:33:19.535000+00:00,NaN,NaN,NaN,0.311828,...,8,6,0.01,0.01,None,LOCAL,admin,"[{""run_id"": ""f77ef7565cc24c7d8a590f7af3c55a7a""...",eta_0.01_subsample_0.5_max_depth_6_gamma_0.01_...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
1153,3e219404cadd436d9addf214d71f97af,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/3e219404c...,2024-01-17 09:33:08.484000+00:00,2024-01-17 09:33:13.866000+00:00,NaN,NaN,NaN,0.311828,...,6,6,0.01,0.01,None,LOCAL,admin,"[{""run_id"": ""3e219404cadd436d9addf214d71f97af""...",eta_0.01_subsample_0.5_max_depth_6_gamma_0.01_...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
1154,c7c39a8199074c139937bcaa4ddf5a00,634593513755006234,FINISHED,mlflow-artifacts:/634593513755006234/c7c39a819...,2024-01-17 09:33:03.360000+00:00,2024-01-17 09:33:08.418000+00:00,NaN,NaN,NaN,0.311828,...,4,6,0.01,0.01,None,LOCAL,admin,"[{""run_id"": ""c7c39a8199074c139937bcaa4ddf5a00""...",eta_0.01_subsample_0.5_max_depth_6_gamma_0.01_...,C:\Users\admin\anaconda3\Lib\site-packages\ipy...
